In [ ]:
# Install necessary libraries
!pip install datasets transformers huggingface_hub evaluate optuna
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import os
from huggingface_hub import login as hf_login
import subprocess
import wandb
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, AutoTokenizer, AutoModel, RobertaConfig
import evaluate
from sklearn.model_selection import train_test_split
import yaml

We setup the automatic login for Hugging Face and Weights and Biases with the secret token and API key.

In [ ]:
# Auto-login for Hugging Face and Weights & Biases

hf_token = os.environ.get("HF_TOKEN")
wandb_api_key = os.environ.get("WANDB_API_KEY")


hf_login(token=hf_token)
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

wandb.login(key=wandb_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: so-907 (so-907-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Let's load the training and evaluation dataset and tokenize the comments.

In [ ]:
train_dataset = pd.read_csv("train.csv")
eval_dataset = pd.read_csv("test.csv")

# Turn into dataset object
train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

# Instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base", num_labels=3)

def tokenize_function(examples):
  return tokenizer(examples["Comment"],
                   padding="max_length",
                   truncation=True,
                   max_length=512
                   )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/14299 [00:00<?, ? examples/s]

Map:   0%|          | 0/3575 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
  load_accuracy = evaluate.load("accuracy")
  load_f1 = evaluate.load("f1")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
  f1 = load_f1.compute(predictions=predictions, references=labels, average="macro")["f1"]

  return {"accuracy": accuracy, "f1": f1}

Now we load the configuration of the best model that we had saved on Weights & Biases and instantiate a new model with the same parameters.

In [ ]:
def unwrap_values(d):
    if isinstance(d, dict):
        if "value" in d and len(d) == 1:
            return unwrap_values(d["value"])
        else:
            return {k: unwrap_values(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [unwrap_values(x) for x in d]
    else:
        return d

# Load yaml file
with open("config.yaml", "r") as f:
    config_dict = yaml.safe_load(f)

# Clean up the dictionary
config_dict = unwrap_values(config_dict)

# Create configuration
config = RobertaConfig(**config_dict)

# Create model
model = AutoModel.from_pretrained(
    "roberta-base",
    config=config
)

def model_init():
    return AutoModel.from_pretrained(
    "roberta-base",
    config=config
)

Now we save the model as an artifact on Weights & Biases.

In [ ]:
model.save_pretrained("youtube-roberta-base")
tokenizer.save_pretrained("youtube-roberta-base")

wandb.init(project="huggingface", name="roberta-base-replica")

artifact = wandb.Artifact("youtube-roberta-base", type="model")
artifact.add_dir("youtube-roberta-base")
wandb.log_artifact(artifact)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: Adding directory to artifact (./youtube-roberta-base)... Done. 9.5s


<Artifact youtube-roberta-base>